<a href="https://colab.research.google.com/github/aknip/Local_LLMs/blob/main/Mixtral_8x7B_llama.cpp_on_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Approach: Mixtral-8x7B-Instruct via llama.cpp-python

Google T4:
- mixtral-8x7b: 4 token / sec
- mistral-7b: 12 token /sec

In [ ]:
model = 'mixtral-8x7b-instruct-v0.1.Q2_K'
#model = 'openhermes-2-mistral-7b.Q5_K_M'

if model == 'mixtral-8x7b-instruct-v0.1.Q2_K':
	!wget https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q2_K.gguf
elif model == 'openhermes-2-mistral-7b.Q5_K_M':
	!wget https://huggingface.co/TheBloke/OpenHermes-2-Mistral-7B-GGUF/resolve/main/openhermes-2-mistral-7b.Q5_K_M.gguf
elif model == 'dolphin-2.6-mixtral-8x7b.Q2_K':
	!wget https://huggingface.co/TheBloke/dolphin-2.6-mixtral-8x7b-GGUF/resolve/main/dolphin-2.6-mixtral-8x7b.Q2_K.gguf
elif model == 'mistral-7b-instruct-v0.2.Q8':
	!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q8_0.gguf
elif model == 'dolphin-2.2.1-mistral-7b.Q8':
	!wget https://huggingface.co/TheBloke/dolphin-2.2.1-mistral-7B-GGUF/resolve/main/dolphin-2.2.1-mistral-7b.Q8_0.gguf

In [ ]:
# %%capture
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [ ]:
from llama_cpp import Llama
import ctypes

if model == 'mixtral-8x7b-instruct-v0.1.Q2_K':
	llm =Llama(model_path="/content/mixtral-8x7b-instruct-v0.1.Q2_K.gguf", n_gpu_layers=27) # modify number of layers, dependent on RAM/GPU?
  # https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF#example-llamacpp-command
  # ./main -ngl 35 -m mixtral-8x7b-instruct-v0.1.Q4_K_M.gguf --color -c 2048 --temp 0.7 --repeat_penalty 1.1 -n -1 -p "[INST] {prompt} [/INST]"

elif model == 'openhermes-2-mistral-7b.Q5_K_M':
  llm =Llama(model_path="/content/openhermes-2-mistral-7b.Q5_K_M.gguf", n_gpu_layers=32)
  # https://huggingface.co/TheBloke/OpenHermes-2-Mistral-7B-GGUF#example-llamacpp-command
  # ./main -ngl 32 -m openhermes-2-mistral-7b.Q4_K_M.gguf --color -c 2048 --temp 0.7 --repeat_penalty 1.1 -n -1 -p "<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant"


In [ ]:
import json

In [ ]:
# inference
prompt = "Who are you?"
input_prompt = f"""[INST] <<SYS>>
You are a charismatics, talented, respectful and honest musician. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt} [/INST]"""

output = llm(input_prompt,max_tokens=1024)
print(json.dumps(output, indent=2))
print(output["choices"][0]["text"])

In [ ]:
# inference
prompt = "Generate lyrics for a romantic love song"
input_prompt = f"""[INST] <<SYS>>
You are a charismatics, talented, respectful and honest musician. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>
{prompt} [/INST]"""

output = llm(input_prompt,max_tokens=1024)
print(json.dumps(output, indent=2))
print(output["choices"][0]["text"])

# 2. Approach: Mixtral-8x7B-Instruct via llama.cpp and webserver

**=> access to webserver not working!**

- 4 token / second
- It takes about 10 minutes for everything to be setup, downloaded and loaded.
- Wait until the inference cell outputs "llama server listening at... all slots are idle and system prompt is empty, clear the KV cache." Use the URL from the output of the previous cell that says "Connect using the following address when the server is up."

Uses llama.cpp https://github.com/ggerganov/llama.cpp

Source: https://www.reddit.com/r/LocalLLaMA/comments/18ggkp6/mixtral8x7binstruct_on_free_colab_slow_4ts_but/

In [ ]:
#@title #Choose a Model
model = 'mixtral-8x7b-instruct-v0.1.Q2_K' #@param ['mixtral-8x7b-instruct-v0.1.Q2_K', 'dolphin-2.6-mixtral-8x7b.Q2_K', 'mistral-7b-instruct-v0.2.Q8', 'dolphin-2.2.1-mistral-7b.Q8']


In [ ]:
#@title #Runtime Info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


In [ ]:
#@title # Setup

# takes approx. 5 minutes

%cd /content/
!rm -rf llama.cpp
!git clone --depth 1 https://github.com/ggerganov/llama.cpp.git
%cd llama.cpp
!make LLAMA_CUBLAS=1

if model == 'mixtral-8x7b-instruct-v0.1.Q2_K':
	!wget https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF/resolve/main/mixtral-8x7b-instruct-v0.1.Q2_K.gguf
elif model == 'dolphin-2.6-mixtral-8x7b.Q2_K':
	!wget https://huggingface.co/TheBloke/dolphin-2.6-mixtral-8x7b-GGUF/resolve/main/dolphin-2.6-mixtral-8x7b.Q2_K.gguf
elif model == 'mistral-7b-instruct-v0.2.Q8':
	!wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q8_0.gguf
elif model == 'dolphin-2.2.1-mistral-7b.Q8':
	!wget https://huggingface.co/TheBloke/dolphin-2.2.1-mistral-7B-GGUF/resolve/main/dolphin-2.2.1-mistral-7b.Q8_0.gguf


In [ ]:
#@title # Connect using the following address when the server is up.
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(12345)"))

In [ ]:
#@title # inference
%cd /content/llama.cpp
if model == 'mixtral-8x7b-instruct-v0.1.Q2_K':
	!./server -m mixtral-8x7b-instruct-v0.1.Q2_K.gguf -ngl 27 -c 2048 --port 12345
elif model == 'dolphin-2.6-mixtral-8x7b.Q2_K':
	!./server -m dolphin-2.6-mixtral-8x7b.Q2_K.gguf -ngl 27 -c 2048 --port 12345
elif model == 'mistral-7b-instruct-v0.2.Q8':
	!./server -m mistral-7b-instruct-v0.2.Q8_0.gguf -ngl 35 -c 0 --port 12345
elif model == 'dolphin-2.2.1-mistral-7b.Q8':
	!./server -m dolphin-2.2.1-mistral-7b.Q8_0.gguf -ngl 35 -c 0 --port 12345

